In [84]:
import numpy as np
from scipy import random,linalg
from sklearn import datasets
import matplotlib.pyplot as plt
%matplotlib inline
import hdbscan

In [85]:
"""
A function that creates symmetric Matrix which is posritive semidefinite
"""
def sym_matrix(n,state=3):
    matrixsize=n
    A=random.rand(matrixsize,matrixsize)
    B=np.dot(A,A.transpose())
    for i in range(n):
        B[i,i]=0
    return B


In [86]:
T=sym_matrix(20)

In [93]:
clusterer=hdbscan.HDBSCAN(min_cluster_size=30).fit(T)

In [92]:
clusterer.labels_

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1])

In [106]:
"""
An experiment with min_cluster_size
n: number of tests to be carried out
Starting from min_cluster_size 1
"""
def min_clus(n,T):
    for i in range(n):
        clusterer=hdbscan.HDBSCAN(min_cluster_size=i+2).fit(T)
        print(clusterer.labels_)
        
        
    


In [120]:
clusterer2 = hdbscan.HDBSCAN(cluster_selection_method='eom',min_cluster_size=5, min_samples=5,allow_single_cluster=True).fit(T)

In [121]:
clusterer2.labels_

array([-1, -1, -1, -1,  0,  0, -1, -1, -1, -1, -1,  0,  0, -1, -1, -1, -1,
        0, -1, -1])